# مبدل فایل سرفصل درس 

این برنامه تبدیل فایل سرفصل درسها به صورت مارک داون که قابل درج در گیت‌هاب باشد را انجام می‌دهد.

فرض بر این هست که سرفصل همه درسها در یک فایل مارک‌داون است، این برنامه بخشهای مختلف هر درس از قبیل،‌عنوان، تعداد واحد، نوع درسها و ... را استخراج کرده و مشخصات ار درس را در یک فایل مارک‌داون قرار می‌دهد.
سپس از میک‌داکس برای ایجاد صفحات مربوطه استفاده می‌شود
ظاهرا فقط اولی جدول ابتدای هر درس رو به مارک‌داون تبدیل می‌کنه.
باقی اچ‌تی‌ام‌ال می‌کنن.

https://products.aspose.app/total/conversion

https://products.groupdocs.app/conversion/docx-to-md

## M. Amintoosi

In [1]:
# !pip install markdown-strings
# !pip install mkdocs

In [2]:
import io
import re
import markdown_strings
import pandas as pd


In [3]:
def extract_elements(input_str):
    start_idx = input_str.find('|',1)
    start_idx = input_str.find('|',start_idx) # Move to next |
    end_idx = input_str.find('|',start_idx+1)
    prreq = input_str[start_idx+1:end_idx]
    prreq = re.sub('\*', '', prreq)
    prreq = prreq.strip()
    if prreq == '':
        prreq = 'ندارد'
    # print(prreq)

    # نوع درس
    start_idx = end_idx
    end_idx = input_str.find('|',start_idx+1)
    c_cat = input_str[start_idx+1:end_idx]
    c_cat = re.sub('\*', '', c_cat)
    if 'n' in c_cat  or '¢' in c_cat:
        c_cat = c_cat.replace('n','').strip()
        c_cat = c_cat.replace('¢','').strip()
    elif 'حل تمرین' in c_cat:
        c_cat = re.sub('حل تمرین','',c_cat)
    else:
        c_cat = ''
    # print(c_cat)

    # نوع واحد
    start_idx = end_idx
    end_idx = input_str.find('|',start_idx+1)
    unit_type = input_str[start_idx+1:end_idx]
    unit_type = re.sub('\*', '', unit_type)
    if 'n' in unit_type or '¢' in unit_type:
        unit_type = unit_type.replace('n','').strip()
        unit_type = unit_type.replace('¢','').strip()
    else:
        unit_type = ''
    # print(unit_type)
    return prreq, c_cat, unit_type

In [4]:
endinglist = ["abc", "df", "ghi"]
line = "This is a line containing def"

contains_ending = any(ending in line for ending in endinglist)
print(contains_ending)

False


In [5]:
def extract_parts(file_content, line_no, ending_list):
    part_content =[]
    line_no += 1
    line = file_content[line_no]
    while line_no<len(file_content) and not any(ending in line for ending in ending_list):
        if line.strip() != '':
            part_content.append(line)
        line_no += 1
        if line_no>= len(file_content):
            break
        line = file_content[line_no]
    return part_content, line_no

In [6]:
def extract_courses(file_path):
    # pattern = re.compile(r'عنوان درس به فارسی:(.*?)', re.UNICODE)
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
        # courses = pattern.findall(data)
        courses = re.findall(r'عنوان درس به فارسی:(.*?)عنوان درس به فارسی:', data, re.DOTALL)#flags=re.UNICODE)
        # courses = re.findall(r'Stewart(.*?)Stewart', data,  re.DOTALL)#flags=re.UNICODE)
        return courses


In [7]:
# !dir

In [8]:
# استفاده از تابع برای استخراج اطلاعات درس‌ها
file_path = '../input/CS-Chart.md'
extracted_courses = extract_courses(file_path)

# نمایش نتایج
# for course in extracted_courses:
#     print(course)

In [9]:
len(extracted_courses)
# , extracted_courses

40

In [10]:
# Load the file into file_content
file_content = [line for line in open('../input/CS-Chart.md')]
file_content = [line.replace('**منابع**:', '**منابع:**') for line in file_content]
col_names = ['fa_title','en_title','en_file_name','c_cat','complementary','prreq','coreq',\
                           'c_unit','unit_type','c_hours','c_ex','c_goal1','c_goal2','sar_fasl','c_refs']
df = pd.DataFrame(columns=col_names)

fa_col_names = ['عنوان درس به فارسی:', 'عنوان درس به انگلیسی:', 'نوع درس:', 'آموزش تکمیلی:','پیش‌نیاز:','هم‌نیاز',\
                 'تعداد واحد:', 'نوع واحد:', 'تعداد ساعت:','حل تمرین:', 'اهداف درس:', 'توانایی و شایستگی‌هایی که درس پرورش می‌دهد:',\
                      'سرفصل درس:','منابع:']
# df.set_index('fa_title',inplace = True)
line_no = 0
# line = file_content[line_no]
# while line_no < len(file_content) and line.find(fa_col_names[0]) == -1:
#     line_no += 1
#     line = file_content[line_no]
# # line_no -=1

while line_no < len(file_content):
    line = file_content[line_no]
    pattern = r"\| --- \|"
    result = re.sub(pattern, "", line)
    # pattern = r"\|"
    # result = re.sub(pattern, "", result)
    srch_str = "عنوان درس به فارسی:"
    # print(result, result.find(srch_str))
    if result.find(srch_str) != -1:
        result = re.sub("<a.*?/a>",'',result)
        start_idx = result.find('<h3>') + 4
        end_idx = result.rfind('</h3>')
        fa_title = result[start_idx:end_idx]
        fa_title = re.sub('\*', '', fa_title)
        fa_title = fa_title.strip()
        line_no += 1
        print(fa_title)
        continue

    # line_no += 2 # for | -: | :- |

    srch_str = "عنوان درس به انگلیسی:"
    # print(result , result.find(srch_str))
    if result.find(srch_str) != -1:
        start_idx = result.find('|',1)
        end_idx = result.find('|',start_idx+1)
        en_title = result[start_idx+1:end_idx]
        en_title = re.sub('\*', '', en_title)
        en_title = re.sub('<.*?>', '', en_title)
        en_title = en_title.strip()
        en_file_name = re.sub('\&', 'and', en_title)
        en_file_name = re.sub('\n', '', en_file_name)
        en_file_name = re.sub(' ', '-', en_file_name)
        # print(en_title)

        # پیش نیاز
        line_no += 1
        line = file_content[line_no]
        prreq, c_cat, unit_type = extract_elements(line)
        # print(prreq, c_cat, unit_type)

        line_no += 1
        line = file_content[line_no]
        coreq, c_cat_, unit_type_ = extract_elements(line)
        # if c_cat_ != '':
        #     print('\n', c_cat_, '\n')
        c_cat, unit_type = c_cat+c_cat_,  unit_type+unit_type_
        # print(coreq, c_cat, unit_type)

        line_no += 1
        line = file_content[line_no]
        c_unit, c_ex, c_cat_ = extract_elements(line)
        if unit_type == '':
            unit_type = "نظری-عملی"
        c_cat = c_cat+c_cat_
        # print(c_unit, c_ex, c_cat)

        line_no += 1
        line = file_content[line_no]
        c_hours, _, _ = extract_elements(line)
        print(c_hours, '\n')
       
        # common_ending_list = ['برنامه‌ی هشت ترمی','دروس تخصصی', 'دروس الزامی']

        while True:
            line_no += 1
            if line_no >= len(file_content):
                break
            line = file_content[line_no]
            if line.find(fa_col_names[-2]) != -1: #سرفصل
                break
        if line_no >= len(file_content):
                break

        # continue
        # خواندن سرفصل
        
        # ending_list = common_ending_list.copy()
        # ending_list.append(fa_col_names[-1])
        # sar_fasl, line_no = extract_parts(file_content, line_no, ending_list)

        sar_fasl =[]
        line_no += 1
        line = file_content[line_no]
        while line_no<len(file_content) and line.find(fa_col_names[-1]) == -1: #refs
            if line.strip() != '':
                sar_fasl.append(line)
            line_no += 1
            if line_no>= len(file_content):
                break
            line = file_content[line_no]

        if line_no >= len(file_content):
            break    

        # ending_list = common_ending_list.copy()
        # ending_list.append('عنوان درس به فارسی')
        # refs, line_no = extract_parts(file_content, line_no, ending_list)
        refs = []
        line_no += 1
        line = file_content[line_no]
        while line_no<len(file_content) and line.find("عنوان درس به فارسی:") == -1:
            if line.strip() != '':
                refs.append(line)
            line_no += 1
            if line_no>= len(file_content):
                break
            line = file_content[line_no]
        #     # print(line_no)

        if line.find("عنوان درس به فارسی:") != -1:
            line_no -= 1
        this_course ={'fa_title':fa_title, 'en_title':en_title, 'en_file_name':en_file_name,'c_cat':c_cat,'prreq':prreq,'coreq':coreq,\
                           'c_unit':c_unit,'unit_type':unit_type,'c_hours':c_hours,'c_ex':c_ex,\
                            'sar_fasl':sar_fasl,'c_refs':refs}
        df = df.append(this_course,ignore_index=True)
    line_no += 1
    # print('')
    
df.head(7)

ریاضی عمومی ۱
64 

ریاضی عمومی ۲
64 

معادلات دیفرانسیل
48 

مبانی کامپیوتر و برنامه‌سازی
48 

مبانی علوم ریاضی
48 

آمار و احتمال ۱
48 

نرم‌افزارهای آماری و تحلیل داده‌ها
32 

کارگاه کامپیوتر ۱
32 

برنامه‌سازی پیشرفته
48 

زبان‌های برنامه‌سازی
48 

ساختمان داده‌ها
48 

طراحی و تحلیل الگوریتم‌ها
48 

کارگاه کامپیوتر ۲
32 

پایگاه داده‌ها
48 

اصول سیستم‌های کامپیوتری
48 



مبانی ترکیبیات
64 

مبانی منطق و نظریه مجموعه‌ها
48 

مبانی نظریه محاسبه
48 

مبانی محاسبات علمی
48 

مبانی آنالیز عددی
48 

مبانی آنالیز ریاضی
48 

تحلیل آماری داده‌ها
48 

سری‌های زمانی کاربردی
48 

یادگیری ماشین مقدماتی
48 

داده‌کاوی مقدماتی
48 

تحقیق در عملیات
48 

بهینه‌سازی غیرخطی
48 

مبانی اقتصاد دیجیتال
32 

کارآموزی
192 

کاربینی
32 

کارآفرینی
32 

هوش مصنوعی
48 

آشنایی با یادگیری عمیق
48 

بهینه‌سازی گسسته
48 

آشنایی با نظریه بازی‌ها
48 

آمار و احتمال ۲
48 

آمار محاسباتی
48 

روش‌های آماری
48 

نظریه محاسبه
48 

منطق برای علوم کامپیوتر
48 

نظریه مقدماتی کدگذاری
48 

رمزنگاری
48 

برنامه‌نویسی امن
48 

معناشناسی عملیاتی برنامه‌نویسی
48 

الگوریتم‌های تصادفی
48 

هندسه محاسباتی
48 

رگرسیون ۱
48 

هوش محاسباتی
48 

هوش تجاری مقدماتی
48 

تجارت الکترونیک
32 

مدلسازی ریاضی
48 

آزمایشگاه ریاضی
32 

ریاضیات فازی
48 

نظریه گراف و کاربردها
48 

مبانی جبر
48 

مبانی آنالیز فوریه و موجک‌ها
48 

پردازش تصویر مقدماتی
48 

برنامه‌نویسی موبایل
48 

برنامه‌نویسی وب
48 

شبکه‌های

,fa_title,en_title,en_file_name,c_cat,complementary,prreq,coreq,c_unit,unit_type,c_hours,c_ex,c_goal1,c_goal2,sar_fasl,c_refs
0,ریاضی عمومی ۱,Calculus I,Calculus-I,پایه,NaN,ندارد,ندارد,4,نظری,64,دارد,NaN,NaN,[مختصات دکارتی، مختصات قطبی، اعداد مختلط و جمع...,"[-\n, - - استوارت، ج. (1395). حساب دیفرانسیل و..."
1,ریاضی عمومی ۲,Calculus II,Calculus-II,پایه,NaN,ریاضی عمومی ۱,ندارد,4,نظری,64,دارد,NaN,NaN,[فضای اقلیدسی، معادلات پارامتری، مختصات فضائی،...,"[-\n, - - استوارت، ج. (1395). حساب دیفرانسیل و..."
2,معادلات دیفرانسیل,Differential Equations,Differential-Equations,پایه,NaN,ریاضی عمومی ۱,ندارد,3,نظری,48,دارد,NaN,NaN,[مفاهیم و تعاریف اولیه .معادلات جداپذیر و معاد...,"[-\n, - - ظهوری زنگنه، ح. (مترجم). (1394). معا..."
3,مبانی کامپیوتر و برنامه‌سازی,Fundamentals of Computer Programming,Fundamentals-of-Computer-Programming,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[آشنایی مقدماتی با ساختار کامپیوتر، معرفی کلی ...,"[-\n, - - Horton, I. (2013). Beginning C (Expe..."
4,مبانی علوم ریاضی,Foundation of Mathematics,Foundation-of-Mathematics,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[منطق گزاره ها و جدول ارزش. مجموعه‌ها و اعمال ...,"[-\n, - - موحد، ضیاء (1379). درآمدی به منطق جد..."
5,آمار و احتمال ۱,Probability and Statistics I,Probability-and-Statistics-I,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[احتمال: فضای احتمال، قوانین احتمال، مدل های ا...,"[-\n, - - راس، شلدون (1389). مبانی احتمال، ویر..."
6,نرم‌افزارهای آماری و تحلیل داده‌ها,Statistical Software & data analysis,Statistical-Software-and-data-analysis,تخصصی الزامی,NaN,ندارد,ندارد,1,عملی,32,,NaN,NaN,[تعریف علم آمار، بیان کاربردهای آمار در سایر ع...,"[-\n, - - گوس، جی و منیروپ، د. (1401). آمار با..."


In [11]:
# df.head(7)

In [12]:
# # می‌خواستم کاراکتر خط جدید از جدول روش یاددهی رو حذف کنم. چون در خروجی یک خط فاصله می‌افتاد و جدول نشون داده نمیشه  
# # نام ستونی که روی عناصر آن حرکت می‌کنیم
# column_name = "sar_fasl"

# # حلقه بر روی عناصر ستون خاص
# for i in range(len(df)):
#     sar_fasl = df.loc[i, column_name]
#     print(sar_fasl)
#     # حذف از بخش روش یاددهی به بعد
#     index = None
#     new_sar_fasl = []
#     for j, item in enumerate(sar_fasl):
#         if item.startswith('|'):
#             new_sar_fasl.append(item.strip())
#         else:
#             new_sar_fasl.append(item)
#     print(new_sar_fasl)
#     df.at[i, column_name] = new_sar_fasl


In [13]:
# df
# df['sar_fasl'][10]

In [14]:
# می‌خواستم از بخش روش یاددهی به بعد که در مراحل قبل به عنوان سرفصل خوانده شده رو حذف کنم
# نام ستونی که روی عناصر آن حرکت می‌کنیم
column_name = "sar_fasl"

# حلقه بر روی عناصر ستون خاص
for i in range(len(df)):
    sar_fasl = df.loc[i, column_name]
    print(sar_fasl)
    # حذف از بخش روش یاددهی به بعد
    index = None
    for j, item in enumerate(sar_fasl):
        if "روش یاددهی-یادگیری" in item:
            index = j
            break

    # حذف عناصر از ایندکس  به بعد
    if index is not None:
        sar_fasl = sar_fasl[:index]
        # print('After prune:', type(sar_fasl))
        df.at[i, column_name] = sar_fasl
# A = df['sar_fasl'][20]

df[column_name][10]

['مختصات دکارتی، مختصات قطبی، اعداد مختلط و جمع و ضرب و ریشه آنها، نمایش های مختلف اعداد مختلط، دنباله های عددی، حد و قضایای مربوط، حد بینهایت و حد در بینهایت، حد چپ و راست،  پیوستگی، پیوستگی دنباله ای، قضیه مقدار میانی برای توابع پیوسته، قضیه مقدار اکسترمم برای توابع پیوسته، مشتق، دستورهای مشتق گیری، تابع معکوس و مشتق آن، مشتق توابع مثلثاتی و توابع معکوس آنها، قضیه مشتق ترکیب توابع، قضیه فرما (صفر شدن مشتق در یک نقطه اکسترمم)، قضیه رل، قضیه مقدار میانگین، قضیه مقدار میانگین کشی، آزمون مشتق اول و دوم برای اکسترمم ها، تقعر منحنی، نقطه عطف، دیفرانسیل یک تابع، بسط تیلر، کاربردهای هندسی و فیزیکی مشتق، خم ها، سرعت و شتاب در مختصات قطبی، کاربرد مشتق در تقریب ریشه های معادلات، تعریف انتگرال توابع و انتگرال پذیری، قضیه مقدار میانگین برای انتگرال ها، قضایای اساسی حساب دیفرانسیل و انتگرال، تابع اولیه، روش های تقریبی برآورد انتگرال، کاربرد انتگرال در محاسبه مساحت و حجم و طول منحنی و ...، لگاریتم و تابع نمایی و مشتق آنها، تابع های هذلولوی، روش های انتگرال گیری (همه روش ها)، آشنایی با تابع گاما و خ

['آنالیز پیچیدگی زمان و حافظه الگوریتم\u200cها، توابع بازگشتی، معرفی ساختمان\u200cهای داده\u200cای مقدماتی (لیست پیوندی، پشته، صف و صف اولویت\u200cدار) و الگوریتم\u200cهای وابسته به آن\u200cها،  نمایش ماتریس\u200cها در حافظه، ماتریس\u200cهای تنک،\u200c تطابق رشته، درخت\u200cها و الگوریتم\u200cهای پیمایش مربوطه، درخت عبارت، تبدیل نگارش\u200cهای مختلف یک عبارت ریاضی، درهم\u200cسازی، گراف\u200cها و الگوریتم\u200cهای مربوط به گراف (BFS، DFS) ، الگوریتم\u200cهای جستجو و مرتب\u200cسازی.\n']

In [15]:
def table_from_rows(table_list):
    """Return a formatted table, using each list as the list. The specifics are
    the same as those for the table function.

    >>> table_from_rows([["1","2","3"],["4","5","6"],["7","8","9"]])
    '| 1 | 2 | 3 |\\n| --- | --- | --- |\\n| 4 | 5 | 6 |\\n| 7 | 8 | 9 |'
    """
    # transpose the list
    number_of_rows = len(table_list[0])
    transposed = []
    for column_number in range(0, number_of_rows):
        column_list = [row[column_number] for row in table_list]
        transposed.append(column_list)

    return markdown_strings.table(transposed)


In [16]:
df.tail(2)

,fa_title,en_title,en_file_name,c_cat,complementary,prreq,coreq,c_unit,unit_type,c_hours,c_ex,c_goal1,c_goal2,sar_fasl,c_refs
79,آشنایی با پردازش زبان طبیعی,Introdution to Natural Language Processing,Introdution-to-Natural-Language-Processing,تخصصی اختیاری,NaN,هوش مصنوعی، یادگیری ماشین مقدماتی,ندارد,3,نظری,48,دارد,NaN,NaN,[معرفی پردازش زبان طبیعی، تاریخچه مختصر، مسائل...,"[-\n, - - Jurafsky, D., & Martin, J. H. (2023)..."
80,مهارت‌های نرم شغلی,Job Soft Skills,Job-Soft-Skills,تخصصی اختیاری,NaN,کاربینی,ندارد,2,نظری,32,,NaN,NaN,[-تعریف مهارت‌های نرم و تفاوت آن با مهارت‌های ...,[منابع آزاد با نظر استاد درس (سایت متمم توصیه ...


شناسایی دسته هر درس، مشخص کردن نام فایل درس

In [17]:
for index, row in df.iterrows():
    # print(row['fa_title'], row['en_title'])
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
    # print(r_path)
        # 20240122
    # lesson_name_en = row['en_title'].strip()
    # lesson_name_en = re.sub('\&', 'and', lesson_name_en)
    # lesson_name_en = re.sub(' ', '-', lesson_name_en)
    # lesson_file_name = r_path+'/'+lesson_name_en+'.md'
    lesson_file_name = r_path+'/'+row['en_file_name']+'.md'
    row['en_file_name'] = lesson_file_name
    
df.tail(2)

,fa_title,en_title,en_file_name,c_cat,complementary,prreq,coreq,c_unit,unit_type,c_hours,c_ex,c_goal1,c_goal2,sar_fasl,c_refs
79,آشنایی با پردازش زبان طبیعی,Introdution to Natural Language Processing,elective/Introdution-to-Natural-Language-Proce...,تخصصی اختیاری,NaN,هوش مصنوعی، یادگیری ماشین مقدماتی,ندارد,3,نظری,48,دارد,NaN,NaN,[معرفی پردازش زبان طبیعی، تاریخچه مختصر، مسائل...,"[-\n, - - Jurafsky, D., & Martin, J. H. (2023)..."
80,مهارت‌های نرم شغلی,Job Soft Skills,elective/Job-Soft-Skills.md,تخصصی اختیاری,NaN,کاربینی,ندارد,2,نظری,32,,NaN,NaN,[-تعریف مهارت‌های نرم و تفاوت آن با مهارت‌های ...,[منابع آزاد با نظر استاد درس (سایت متمم توصیه ...


In [18]:
# df.tail(5)

In [19]:
def prreq_links(prreq_str, path_mod = ''):
# تابعی برای استخراج نام هر درس از لیست درسهای پیش‌نیازی و درج لینک فایل مربوطه
    # prreq_str = "مبانی آنالیز ریاضی، مبانی ماتریس‌ها و جبر خطی "
    p_strs = prreq_str.split('،')
    p_list = []
    for p in p_strs:
        x = df[df['fa_title'].str.contains(p.strip())][['fa_title','en_file_name']]
        p_str = p
        if not x.empty:
            fa_title, en_file_name = x['fa_title'].values[0], x['en_file_name'].values[0]
            p_str = '['+fa_title+']('+path_mod+en_file_name+')'
        p_list.append(p_str)
    return ' ،'.join(p_list)


حذف فایلهای پوشه‌ها

In [20]:
import os
def clear_doc_folders(docs_path):
    for path in ['base', 'elective', 'mandatory']:
        directory = os.path.join(docs_path,path)
        # تمام فایل‌ها و زیرمجموعه‌های آن را بدست آورید
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                # بررسی حجم فایل و نام فایل
                # os.path.getsize(file_path) != 0 and
                if  file != '_category_.json':
                    # حذف فایل
                    os.remove(file_path)

In [21]:
doc_folders = './docs/curriculum/'
clear_doc_folders(doc_folders)

ایجاد فایل برای تک تک درسها با فرمت چپ به راست

In [22]:
# ref_file_name = 'output/refs.txt'
# ref_writer = open(ref_file_name, 'w')
for index, row in df.iterrows():
    lesson_file_name = doc_folders+row['en_file_name']
    lesson_writer = open(lesson_file_name, 'w')
    md_str = markdown_strings.header(row['fa_title']+'\n', 1)
    lesson_writer.write(md_str)
    md_str = markdown_strings.header(row['en_title']+'\n', 2)
    lesson_writer.write(md_str)
    lesson_writer.write(markdown_strings.horizontal_rule())
    lesson_writer.write('\n')
    lesson_writer.write(table_from_rows([\
    ["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"],\
    ["پیش‌نیاز:",prreq_links(row['prreq'],path_mod = '../'),"گروه درس:",row['c_cat']],\
    ["هم‌نیاز:",row['coreq'],"نوع درس:",row['unit_type']],\
    ["تعداد واحد:",row['c_unit'],"تعداد ساعت:",row['c_hours']],\
    ["حل تمرین:",row['c_ex'],"",""]\
    ]))
    lesson_writer.write('\n\n')
    lesson_writer.write('**سرفصل درس:**\n\n')
    tmp_lines = [re.sub('-\n', '', rf) for rf in row['sar_fasl']]
    tmp_lines = [re.sub('- -', '-', rf) for rf in tmp_lines]
    lesson_writer.write("\n".join(tmp_lines))
    # lesson_writer.write("\n".join(row['sar_fasl']))
    # for line in row['sar_fasl']:
    #     lesson_writer.write(f"{line}\n")
    lesson_writer.write('\n**منابع:**\n\n')
    tmp_lines = [re.sub('-\n', '', rf) for rf in row['c_refs']]
    tmp_lines = [re.sub('- -', '-', rf) for rf in tmp_lines]

    # Correct the links using Regex
    tmp_lines = [re.sub(r'<(url:)?((http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-]))>', r'[\2](\2)', rf) for rf in tmp_lines]

    # Remove lines that contains `.png` links
    tmp_lines = [rf for rf in tmp_lines if '.png' not in rf]
    
    lesson_writer.write("\n".join(tmp_lines))
    lesson_writer.close()

#     ref_writer.write("\n\n"+index.__str__()+" - "+ row['fa_title']+"\n")
#     refs = [re.sub('\*', '', rf) for rf in row['c_refs']]
#     # print(refs)
#     ref_writer.write("".join(refs))

# ref_writer.close()


نمی دونم درس مبانی نظریه محاسبه چرا وقتی به عنوان پیش نیاز در درسهای دیگر مانند کامپایلر میاد
لینکش درست نیست!

In [26]:
df[df['en_file_name'].str.contains('Computation')]

,fa_title,en_title,en_file_name,c_cat,complementary,prreq,coreq,c_unit,unit_type,c_hours,c_ex,c_goal1,c_goal2,sar_fasl,c_refs
17,مبانی نظریه محاسبه,Introduction to the theory of Computation,mandatory/Introduction-to-the-theory-of-Comput...,تخصصی الزامی,NaN,مبانی کامپیوتر و برنامه‌سازی,ندارد,3,نظری,48,ندارد,NaN,NaN,"[-\n, - اتاماتای متناهی به عنوان یک ماشین محاس...","[-\n, - - Sipser, M. (2012). Introduction to t..."
36,آمار محاسباتی,Computational Statistics,elective/Computational-Statistics.md,تخصصی اختیاری,NaN,آمار و احتمال ۱,ندارد,3,نظری,48,,NaN,NaN,[مروری بر مباحث آمار توصیفی، پالایش داده‌ها با...,"[-\n, - - گیونز، جف‌ اچ‌. (۱۳۹۳). آما‌ر محا‌سب..."
38,نظریه محاسبه,Theory of Computation,elective/Theory-of-Computation.md,تخصصی اختیاری,NaN,مبانی نظریه محاسبه,ندارد,3,نظری,48,ندارد,NaN,NaN,"[-\n, - معرفی انواع مدل های محاسباتی تورینگ که...","[-\n, - - Sipser, M. (2003). Introduction to t..."
45,هندسه محاسباتی,Computational Geometry,elective/Computational-Geometry.md,تخصصی اختیاری,NaN,طراحی و تحلیل الگوریتم‌ها,ندارد,3,نظری,48,دارد,NaN,NaN,"[-\n, - پوسته‌ی محدب. محاسبه‌ی پوسته‌ی محدب در...","[-\n, - - دی برگ، م.، چیونگ، اوتفرید، وان کریو..."
47,هوش محاسباتی,Computational Intelligence,elective/Computational-Intelligence.md,تخصصی اختیاری,NaN,برنامه‌سازی پیشرفته,ندارد,3,نظری,48,: دارد,NaN,NaN,[**روشهای فازی**: مقدمه، مبانی نظری مجموعه‌ها...,"[-\n, - - Keller, J. M., Liu, D., & Fogel, D. ..."
